# E6 - Web Scraping

## E6.1 Hvorfor web scraping?

Det er mange nettsider som ikke gir tilgang til sine data gjennom APIer. Hvis man alikevel ønsker å laste ned dataene for videre prosessering kan man benytte seg av web scraping. For å gjøre web scraping må man skrive kode for nedlastning og parsing av websider og tilhørende ressurser. I python kan man gjøre dette med bibliotekene requests og BeautifulSoup4. Biblioteket requests tar seg av nedlastning, og BeautifulSoup4 tar seg av parsingen av html. Noen ressurser, som feks script, krever at man skriver regulære-uttrykk for å hente ut nødvendig informasjon.

Det må sies at web scraping bør gjøres på en god og hensiktsmessig måte. Du har selv et ansvar for å begrense last på nettsiden og respektere opprettshav for informasjonen du laster ned. De fleste nettsider i dag har en robots.txt hostet under root-folderen. Man bør ta en titt på denne filen før man scraper en nettside.

## E6.2 Verktøy
Når man skal scrape en nettside bør man starte med å gjøre seg kjent med nettsidens struktur og ressurser. Dette kan enkelt gjøres i nettleseren sin utvikler-verktøy. I browserens utvikler-verktøy kan man utforske en nettsides struktur ved å søke og se på nettverkstrafikk samt hvilke filer som er lastet ned.

I firefox og chrome får man tilgang til utvikler-verktøy ved å trykke F12. 'Inspector' gir mulighet for å utforske html. 'Network' gir tilgang til nettverkstrafikk. Av og til må man laste siden på nytt for å få opp trafikken. Under 'Storage' kan man se hvilke ressurser som er lastet ned.

## E6.3 Et enkelt eksempel

In [1]:
import requests               # https://requests.readthedocs.io/en/latest/
from bs4 import BeautifulSoup # https://www.crummy.com/software/BeautifulSoup/bs4/doc/

def download_webpage(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

def extract_data(webpage):
    soup = BeautifulSoup(webpage)
    # print(page.prettify())

    data = []
    
    # Legg merke til bruk av class_ istedenfor class. Konvensjonell nødvendighet.
    for match in soup.find_all(class_="col-md-4 country"):
        country_name = list(match.find(class_="country-name").children)[2].string.strip()
        country_capital = match.find(class_="country-capital").string
        country_population = match.find(class_="country-population").string
        country_area = match.find(class_="country-area").string
        
        data.append([country_name, country_capital, country_population, country_area])
        
    return data

url = "http://www.scrapethissite.com/pages/simple/"
data = extract_data(download_webpage(url))

print(data[:10])



[['Andorra', 'Andorra la Vella', '84000', '468.0'], ['United Arab Emirates', 'Abu Dhabi', '4975593', '82880.0'], ['Afghanistan', 'Kabul', '29121286', '647500.0'], ['Antigua and Barbuda', "St. John's", '86754', '443.0'], ['Anguilla', 'The Valley', '13254', '102.0'], ['Albania', 'Tirana', '2986952', '28748.0'], ['Armenia', 'Yerevan', '2968000', '29800.0'], ['Angola', 'Luanda', '13068161', '1246700.0'], ['Antarctica', 'None', '0', '1.4E7'], ['Argentina', 'Buenos Aires', '41343201', '2766890.0']]


## E6.4 Undersøk robots.txt for nyttig informasjon og begrensninger
[Google create robot.txt](https://developers.google.com/search/docs/crawling-indexing/robots/create-robots-txt)
[Brightdata](https://brightdata.com/blog/how-tos/robots-txt-for-web-scraping-guide)
[Zenrows](https://www.zenrows.com/blog/robots-txt-web-scraping)

Det er ikke alltid at det som står i robots.txt stemmer. Feks kan noen nettsider indikere at alle agenter får lov til å scrape og samtidig blokkere agenter.

### Noen nettsider krever at du later som at du er en nettleser

In [2]:
import requests               # https://requests.readthedocs.io/en/latest/
from bs4 import BeautifulSoup # https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import time

def download_webpage(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

def download_webpage_browser(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0"})
    response.raise_for_status()
    return response.text

url = "https://www.brewersfriend.com/homebrew-recipes/"

for func in [download_webpage, download_webpage_browser]:
    try:
        webpage = func(url)
        print(f"{func}: SUCCESS", flush=True)
    except:
        print(f"{func}: FAILED", flush=True)

    # https://www.brewersfriend.com/robots.txt
    # Crawl-Delay: 20
    if func != download_webpage_browser:
        time.sleep(20)


<function download_webpage at 0x74c3505e2fc0>: FAILED
<function download_webpage_browser at 0x74c3505e2de0>: SUCCESS


### Behov for cache

Når en nettside har en stor begrensning på hvor ofte du kan laste ned nye sider eller ressurser så kan det lønnes seg å cache data i lokalt filsystem. Data som ligger lokalt trenger man ikke å begrense tilgang til. Her finnes det biblioteker man kan benytte seg av.

# E6.5 Oppgaver

Hvis du ikke har nett tilgjengelig så ligger det eksempel-sider under files/scraping

1. Velg en av nettsidene under og lag kode som laster ned og henter ut relevant data fra hovedsiden
  - http://books.toscrape.com/
  - http://quotes.toscrape.com/
2. Implementer rate-limiting med 1 request per 5 sekunder i funksjonen for nedlastning.
3. Implementer en caching-mekanisme som lagrer nettsider som er lastet ned i funksjonen for nedlastning.
   Du trenger en fil med en "url":"data/filenavn" mapping og evt. en mappe med data.
   Biblioteket json er nyttig for å lagre og laste mapping-filen.
4. Utvid oppgave 1 ved å hente ut mer informasjon fra linker assosiert med dataene fra hovedsiden.
   Se etter href-tagen i under-elementer.